In [6]:
%matplotlib widget
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
import matplotlib.pyplot as plt
import lib.simulator as sim

In [7]:
# model procesa - motor
def motor(K, L_a, R_a, Kb, J, k_f, x0):
    def deriv(t, x, u):
        i_a = x[0]
        w = x[1]
        u_b = w*Kb
        di_a = 1/L_a*(u(0)*K-i_a*R_a-u_b)
        M_m = Kb*i_a
        dw = 1/J*(M_m+u(1)-k_f*w)
        return [di_a, dw]

    def out(t, x, u):
        return x[1]
    
    return (deriv, out, x0) 

In [8]:
def P(K):
    return sim.gain(K)

def I(Ti):
    return sim.tf([1],[Ti, 0])

def D(Td, N=10):
    return sim.tf([Td, 0],[Td/N, 1])


In [36]:
# model cjelokupnog sustava

blocks = [
    sim.step(y0=0, y1=1, t_step=1),  #0                  ulazni signal - napon motora
    sim.step(y0=0, y1=1, t_step=6),  #0                  ulazni signal - vanjski moment
    sim.suma('+-'),
    P(K=0.3),
    I(Ti=15),
    D(Td=0.1, N=10),
    sim.suma('+++'),
    motor(K=230, L_a=10, R_a=50, Kb=9, J=0.4, k_f=0.1, x0=[0.0, 0.0]),   # parametri motora
    #sim.tf([1],[1, 3, 3, 1])
    ]

connections = [
    ((0, 0), (2, 0)),
    ((2, 0), (3, 0)),
    ((3, 0), (6, 0)),
    ((3, 0), (4, 0)),
    ((3, 0), (5, 0)),
    ((4, 0), (6, 1)),
    ((5, 0), (6, 2)),
    ((6, 0), (7, 0)),
    ((1, 0), (7, 1)),
    ((7, 0), (2, 1)),
]

outputs = [0, 1, 2, 3, 4, 5, 6, 7]

t, y = sim.simulate(blocks, connections, outputs, t_final=10.0)

In [37]:
# vremenski odziv
fig = plt.figure()
fig.set_label('Odziv procesa')
ax1=plt.subplot(2, 1, 1)
plt.plot(t, y[:,0], t, y[:,7], t, y[:,2])
plt.grid()
plt.legend(['Zadana vrijednost', 'Mjerena vrijednost', 'Reg. odstupanje'])
ax2=plt.subplot(2, 1, 2)
plt.plot(t, y[:,6], t, y[:,3], t, y[:,4], t, y[:,5])
plt.grid()
plt.legend(['Izlaz regulatora', 'P', 'I', 'D'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …